#   IAU Zadanie - II. fáza - COPY

### Autori: Peter Brandajsky - 50%, Frederik Duvač - 50%

# 2.1 Realizácia predspracovania dát

## 2.1.A

## 2.1.B

## 2.1.C

## 2.1.D

# 2.2 Výber atribútov pre strojové učenie

## 2.2.A

## 2.2.B

## 2.2.C

# 2.3 Replikovateľnosť predspracovania

## 2.3.A

## 2.3.B